In [1]:
import dynamic_singer as dsinger

## source

For source, I am going to use fixer-io, https://github.com/huseinzol05/dynamic-singer/tree/master/tap-fixerio

Make sure we already installed the module in our local,

```bash
git clone https://github.com/huseinzol05/dynamic-singer.git
cd dynamic-singer
pip install -e tap-fixerio
```

And if we check in https://github.com/huseinzol05/dynamic-singer/tree/master/tap-fixerio Readme, we need to build a json config,

```json
{
    "base": "USD",
    "start_date": "<YYYY-MM-DD>",
    "access_key": "<your_access_key>",
    "symbols": ["USD","EUR","GBP","JPY"]
}
```

So we need to create one. Just get our `access_token` at https://fixer.io/quickstart

In [2]:
import json

config = {
    'base': 'USD',
    'start_date': '2020-05-05',
    'access_key': '6b3f8a3cd7843530014751f7e6a8066d',
    'symbols': ['GBP']
}

with open('fixer-config.json', 'w') as fopen:
    json.dump(config, fopen)

## start

So to dynamic-singer, we need to initiate `Source` object.

In [3]:
source = dsinger.Source('tap-fixerio --config fixer-config.json')

We are not yet able to run it, because we do not add any data sources to sink. So, let's add one.

## gsheet

For target, I am going to use gsheet, https://github.com/singer-io/target-gsheet, In this example, I am not going to share my gcp secret.

This is the spreadsheet, https://docs.google.com/spreadsheets/d/1fgajBprxaLxVH9GMg25BzD8xYiWCkEKDSxE9Ze28ArY/edit?usp=sharing

Make sure we already installed the module in our local,

```bash
pip3 install target-gsheet
```

In [4]:
config = {
    'spreadsheet_id': '1fgajBprxaLxVH9GMg25BzD8xYiWCkEKDSxE9Ze28ArY'
}
with open('gsheet-config.json', 'w') as fopen:
    json.dump(config, fopen)

In [5]:
source.add('target-gsheet --config gsheet-config.json')

## write to file

Going to create a simple class to save to file.

In [6]:
class Target:
    def __init__(self, filename):
        self.f = open(filename, 'a')
        
    def parse(self, row):
        self.f.write(row)
        return row

In [7]:
target = Target('test.txt')
source.add(target)

## Bigquery

For bigquery target, I am going to use target-bigquery, https://github.com/huseinzol05/dynamic-singer/tree/master/target-bigquery

Make sure we already installed the module in our local,

```bash
git clone https://github.com/huseinzol05/dynamic-singer.git
cd dynamic-singer
pip install -e target-bigquery
```

After that download service account json powerful enough to insert into BQ from GCP IAM and give any name, mine is `bq-account.json`.

And please insert your own `project_id` below.

In [8]:
config = {
    'project_id': '',
    'dataset_id': 'fixerio',
    'key_path': 'bq-account.json',
}

with open('bigquery-config.json', 'w') as fopen:
    json.dump(config, fopen)

In [9]:
source.add('target-bigquery --config bigquery-config.json')

## List targets

In [10]:
source.get_targets()

['target-gsheet --config gsheet-config.json',
 'target-bigquery --config bigquery-config.json']

Here you can delete any targets using `source.delete_target`,

```python
def delete_target(self, index: int):
    """
    Parameters
    ----------
    index: int
        target index from `get_targets()`.
    """
```

## start

In this example, I am going to emit to targets in async manner. Default emitting from first target until last target.

In [11]:
source.start(asynchronous = True)

INFO {"type": "SCHEMA", "stream": "exchange_rate", "schema": {"type": "object", "properties": {"date": {"type": "string", "format": "date-time"}}, "additionalProperties": true}, "key_properties": ["date"]}
INFO None
INFO {"type": "RECORD", "stream": "exchange_rate", "record": {"GBP": "0.871002", "EUR": "1.0", "date": "2020-05-05T00:00:00Z"}}
INFO None
INFO {"type": "RECORD", "stream": "exchange_rate", "record": {"GBP": "0.873037", "EUR": "1.0", "date": "2020-05-06T00:00:00Z"}}
INFO None
INFO {"type": "STATE", "value": {"start_date": "2020-05-06"}}
INFO None


INFO Replicating exchange rate data from fixer.io starting from 2020-05-05
INFO Tap exiting normally
INFO Sending version information to stitchdata.com. To disable sending anonymous usage data, set the config parameter "disable_collection" to true
INFO URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1fgajBprxaLxVH9GMg25BzD8xYiWCkEKDSxE9Ze28ArY?alt=json
INFO URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1fgajBprxaLxVH9GMg25BzD8xYiWCkEKDSxE9Ze28ArY:batchUpdate?alt=json
INFO URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1fgajBprxaLxVH9GMg25BzD8xYiWCkEKDSxE9Ze28ArY?alt=json
INFO URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1fgajBprxaLxVH9GMg25BzD8xYiWCkEKDSxE9Ze28ArY/values/exchange_rate%21A1%3AZZZ:append?valueInputOption=USER_ENTERED&alt=json
INFO URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1fgajBprxaLxVH9GMg25BzD8xYiWCkEKDSxE9Ze28ArY/values/exchange_rate%21A1%3